In [ ]:
# %%

import os
import sys
import re
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import numpy as np
import openpyxl
import pandas as pd
import math
import itertools
import matplotlib.patches as patches
import seaborn as sns
from decimal import Decimal
import glob
import csv
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import sklearn.svm 
import sklearn.metrics
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
import tensorflow as tf
from tensorflow import feature_column
from tensorflow.keras import layers
import pickle

def get_app_list(fileName):
    """get metrics list

    Arguments:
        fileName {string} -- the file name of application file with absolute path

    Returns:
        app {list} -- the app  list
    """
    apps = []
    with open(fileName) as f:
        for line in f.readlines():
            if not line.startswith('#'):
                words = line.strip().split(',')
                app = words[0].strip()
                app_num = words[1].strip()
                apps.append(app+app_num)
        f.close()
    return apps

def load_file(filepath):
    max_len = 1800
    dataframe = pd.read_csv(filepath, index_col=0)
    if (dataframe.shape[1] == 4 and dataframe.shape[0] > 1800):
        if dataframe["u_gpu"].sum() == 0:
            #os.remove(filepath)
            print(filepath)
        else:
            return dataframe.values[:max_len,:]
    else:
        print(filepath)
        
def load_group():
    arch = "p100"
    
    y_label = []
    data_group = []
    category = "mybench"
    pathfolder = '/home/pzou/projects/Power_Signature/results/%s/%s/power-combine'%(category, arch)
    app_list = app_list = get_app_list("/home/pzou/projects/Power_Signature/Scripts/applications_%s.csv"%(category))
    for app in app_list:
        fileName =app+".pwr.csv"
        data = load_file(os.path.join(pathfolder, fileName))
        data_group.append(data)
        y_label.append(0)
    print("normal count", len(data_group))
    
    normal_count =len(data_group)
    category = "risky"
    pathfolder = '/home/pzou/projects/Power_Signature/results/%s/%s/power-combine'%(category, arch)
    app_list = app_list = get_app_list("/home/pzou/projects/Power_Signature/Scripts/applications_%s.csv"%(category))
    for app in app_list:
        fileName =app+".pwr.csv"
        data = load_file(os.path.join(pathfolder, fileName))
        data_group.append(data)
       
        y_label.append(1)
        
    print("risk count", len(data_group)-normal_count)
    data_group = np.asarray(data_group)    
    return data_group, y_label


print("start loading")
data_group, y_label = load_group()
print("Done loading")
#%%

checkpoint_path = "%s/resourceAll-%s.hdf5"%(arch,arch)
model = tf.keras.models.load_model (checkpoint_path)
model